<a href="https://colab.research.google.com/github/V-Vibee/YBI-Internship/blob/main/Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Modules**

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=5db6ae2fbe06d3bc9298193cfe733ad63a9bf64cb441082d48ffd4982ef96411
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark
import pandas as pd
from datetime import date, datetime

**Initiate Spark session**

In [3]:
from pyspark.sql import SparkSession

**Context**

In [4]:
BDS_Session = SparkSession.builder.getOrCreate()

In [5]:
BDS_Session

In [6]:
pandas_df= pd.DataFrame({
    'a':[1,2,3],
    'b':[2., 3., 4.],
    'c':['string1', 'string2', 'string3'],
    'd':[date(2000,1,1), date(2000,2,1), date(2000,3,1)],
    'e':[datetime(2000,1,1,12,0), datetime(2000,1,2,12,0), datetime(2000,1,3,12,0)]
})

In [7]:
pandas_df

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [8]:
type(pandas_df)

pandas.core.frame.DataFrame

In [9]:
df= BDS_Session.createDataFrame(pandas_df)

In [10]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [11]:
type(df)

pyspark.sql.dataframe.DataFrame

In [12]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [13]:
df.columns

['a', 'b', 'c', 'd', 'e']

In [14]:
df.describe().show()

+-------+---+---+-------+
|summary|  a|  b|      c|
+-------+---+---+-------+
|  count|  3|  3|      3|
|   mean|2.0|3.0|   NULL|
| stddev|1.0|1.0|   NULL|
|    min|  1|2.0|string1|
|    max|  3|4.0|string3|
+-------+---+---+-------+



In [15]:
df.select('a', 'b').show()

+---+---+
|  a|  b|
+---+---+
|  1|2.0|
|  2|3.0|
|  3|4.0|
+---+---+



**Converting to Pandas Dataframe**

In [16]:
df.toPandas()

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [17]:
df.filter(df.b==3).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
+---+---+-------+----------+-------------------+



In [18]:
df1= BDS_Session.createDataFrame([
   ['red','banana',1,10], ['blue', 'banana',2,20],
    ['red', 'carrot',3,30],['blue','grape',4,40],
    ['red','carrot',5,50], ['black','carrot',6,60],
    ['red','banana',7,70], ['red', 'grape',8,80] ], schema = ['color', 'fruit', 'v1', 'v2'])

In [19]:
df1.show()


+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [20]:
df1.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



In [21]:
titanic= pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Titanic.csv')

In [22]:
titanic.to_csv('titanic')

In [23]:
titanic= BDS_Session.read.csv('/content/titanic', header=True, inferSchema= True)

In [24]:
titanic.show()

+---+------+--------+--------------------+------+----+-----+-----+--------+--------+-------+--------+----+-----+--------------------+
|_c0|pclass|survived|                name|   sex| age|sibsp|parch|  ticket|    fare|  cabin|embarked|boat| body|           home.dest|
+---+------+--------+--------------------+------+----+-----+-----+--------+--------+-------+--------+----+-----+--------------------+
|  0|     1|       1|Allen, Miss. Elis...|female|29.0|    0|    0|   24160|211.3375|     B5|       S|   2| NULL|        St Louis, MO|
|  1|     1|       1|Allison, Master. ...|  male|0.92|    1|    2|  113781|  151.55|C22 C26|       S|  11| NULL|Montreal, PQ / Ch...|
|  2|     1|       0|Allison, Miss. He...|female| 2.0|    1|    2|  113781|  151.55|C22 C26|       S|NULL| NULL|Montreal, PQ / Ch...|
|  3|     1|       0|Allison, Mr. Huds...|  male|30.0|    1|    2|  113781|  151.55|C22 C26|       S|NULL|135.0|Montreal, PQ / Ch...|
|  4|     1|       0|Allison, Mrs. Hud...|female|25.0|    1|  

#**Regression Problem**

In [25]:
# step 1

! pip install pyspark

In [26]:
# Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark

In [27]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.master('local[3]'). appName('YBIF').getOrCreate()

In [28]:
spark

In [29]:
# Step 2

# Read dataset in pandas
df= pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Boston.csv')
df.to_csv('house')

In [30]:
# Read csv as Sparkdataframe
house= spark.read.csv('/content/house',header=True, inferSchema= True)

In [31]:
house.show()

+---+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+
|_c0|              CRIM|  ZN|INDUS|CHAS|   NX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|
+---+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+
|  0|           0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296.0|   15.3| 396.9| 4.98|24.0|
|  1|           0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242.0|   17.8| 396.9| 9.14|21.6|
|  2|           0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242.0|   17.8|392.83| 4.03|34.7|
|  3|0.0323699999999999| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222.0|   18.7|394.63| 2.94|33.4|
|  4|           0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222.0|   18.7| 396.9| 5.33|36.2|
|  5|           0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222.0|   18.7|394.12| 5.21|28.7|
|  6|           0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311.0|   15.2

In [32]:
# Convert pandas to spark dataframe
house1= spark.createDataFrame(df)

In [33]:
house1.show()

+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+
|              CRIM|  ZN|INDUS|CHAS|   NX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|
+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+
|           0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296.0|   15.3| 396.9| 4.98|24.0|
|           0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242.0|   17.8| 396.9| 9.14|21.6|
|           0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242.0|   17.8|392.83| 4.03|34.7|
|0.0323699999999999| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222.0|   18.7|394.63| 2.94|33.4|
|           0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222.0|   18.7| 396.9| 5.33|36.2|
|           0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222.0|   18.7|394.12| 5.21|28.7|
|           0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311.0|   15.2| 395.6|12.43|22.9|
|           0.14455|

**Define y & X**

In [34]:
# Step 3

house.columns

['_c0',
 'CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT',
 'MEDV']

In [35]:
from pyspark.ml.feature import VectorAssembler

In [36]:
featureassembler= VectorAssembler(inputCols=['CRIM','ZN', 'INDUS','CHAS', 'NX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'], outputCol= 'Features')

In [37]:
output= featureassembler.transform(house)

In [38]:
output.show()

+---+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+--------------------+
|_c0|              CRIM|  ZN|INDUS|CHAS|   NX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|            Features|
+---+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+--------------------+
|  0|           0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296.0|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|  1|           0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242.0|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|  2|           0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242.0|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|  3|0.0323699999999999| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222.0|   18.7|394.63| 2.94|33.4|[0.03236999999999...|
|  4|           0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222.0|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|  5|           

In [39]:
modeldata= output.select('Features', 'MEDV')

In [40]:
modeldata.show()

+--------------------+----+
|            Features|MEDV|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03236999999999...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796000000000...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



In [41]:
# Step 4
train_data, test_data= modeldata.randomSplit([0.8,0.2])

In [42]:
train_data.show()

+--------------------+----+
|            Features|MEDV|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.00905999999999...|32.2|
|[0.01096,55.0,2.2...|22.0|
|[0.01300999999999...|32.7|
|[0.01311,90.0,1.2...|35.4|
|[0.0136,75.0,4.0,...|18.9|
|[0.01381,80.0,0.4...|50.0|
|[0.01432,100.0,1....|31.6|
|[0.01439,60.0,2.9...|29.1|
|[0.01500999999999...|50.0|
|[0.01538,90.0,3.7...|44.0|
|[0.01778,95.0,1.4...|32.9|
|[0.0187,85.0,4.15...|23.1|
|[0.01951,17.5,1.3...|33.0|
|[0.01965,80.0,1.7...|20.1|
|[0.02009,95.0,2.6...|50.0|
|[0.02055,85.0,0.7...|24.7|
|[0.02187,60.0,2.9...|31.1|
|[0.02498,0.0,1.89...|16.5|
|[0.02731,0.0,7.07...|21.6|
+--------------------+----+
only showing top 20 rows



In [43]:
# Step 5: select model

from pyspark.ml.regression import LinearRegression

In [44]:
reg= LinearRegression(featuresCol='Features', labelCol='MEDV')

In [45]:
reg= reg.fit(train_data)

In [46]:
reg.coefficients

DenseVector([-0.1093, 0.0461, 0.0369, 2.8785, -23.456, 3.6215, 0.0047, -1.6981, 0.3138, -0.0122, -0.994, 0.009, -0.5359])

In [47]:
reg.intercept

42.045346933280065

In [48]:
# step 6: Prediction

reg.transform(test_data).show()

+--------------------+----+------------------+
|            Features|MEDV|        prediction|
+--------------------+----+------------------+
|[0.01500999999999...|24.5| 26.85399962653643|
|[0.01709,90.0,2.0...|30.1|24.030867673400582|
|[0.02176999999999...|42.3|  36.8393634783661|
|[0.02543,55.0,3.7...|23.9| 27.43811975583632|
|[0.02729,0.0,7.07...|34.7|30.539533564456256|
|[0.02763,75.0,2.9...|30.8|31.308176433507988|
|[0.02899,40.0,1.2...|26.6|21.235189504246033|
|[0.03048999999999...|31.2|28.000097167477318|
|[0.03113,0.0,4.39...|17.5| 16.03298850678069|
|[0.03358999999999...|34.9| 34.08671567451635|
|[0.03578,20.0,3.3...|45.4| 38.94731675814615|
|[0.03932,0.0,3.41...|22.0|27.784549749148177|
|[0.03961,0.0,5.19...|21.1|20.055768858628863|
|[0.04011,80.0,1.5...|33.3| 35.92687068020239|
|[0.04294,28.0,15....|20.6|27.632126732541337|
|[0.04527,0.0,11.9...|20.6| 22.50319496325102|
|[0.04684,0.0,3.41...|22.6| 27.45069962800183|
|[0.05083,0.0,5.19...|22.2|21.462672280850104|
|[0.05302,0.0

In [49]:
# step 7: Evaluation

y_pred= reg.evaluate(test_data)

In [50]:
y_pred.predictions.show()

+--------------------+----+------------------+
|            Features|MEDV|        prediction|
+--------------------+----+------------------+
|[0.01500999999999...|24.5| 26.85399962653643|
|[0.01709,90.0,2.0...|30.1|24.030867673400582|
|[0.02176999999999...|42.3|  36.8393634783661|
|[0.02543,55.0,3.7...|23.9| 27.43811975583632|
|[0.02729,0.0,7.07...|34.7|30.539533564456256|
|[0.02763,75.0,2.9...|30.8|31.308176433507988|
|[0.02899,40.0,1.2...|26.6|21.235189504246033|
|[0.03048999999999...|31.2|28.000097167477318|
|[0.03113,0.0,4.39...|17.5| 16.03298850678069|
|[0.03358999999999...|34.9| 34.08671567451635|
|[0.03578,20.0,3.3...|45.4| 38.94731675814615|
|[0.03932,0.0,3.41...|22.0|27.784549749148177|
|[0.03961,0.0,5.19...|21.1|20.055768858628863|
|[0.04011,80.0,1.5...|33.3| 35.92687068020239|
|[0.04294,28.0,15....|20.6|27.632126732541337|
|[0.04527,0.0,11.9...|20.6| 22.50319496325102|
|[0.04684,0.0,3.41...|22.6| 27.45069962800183|
|[0.05083,0.0,5.19...|22.2|21.462672280850104|
|[0.05302,0.0

In [51]:
y_pred.meanAbsoluteError

2.9578039076339695

In [52]:
y_pred.meanSquaredError

16.34845241356793

In [53]:
y_pred.rootMeanSquaredError

4.043321952747262

In [54]:
y_pred.r2

0.7836496505729844

Overall model accuracy is 77%

#**Classification**

In [54]:
# step 1

! pip install pyspark

In [54]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.master('local[3]'). appName('YBIF').getOrCreate()

In [55]:
spark

In [56]:
df= pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Diabetes.csv')

In [57]:
# Convert pandas to spark dataframe
diabetes= spark.createDataFrame(df)

In [58]:
diabetes.show()

+-----------+-------+---------+-------+-------+----+------------------+---+--------+
|pregnancies|glucose|diastolic|triceps|insulin| bmi|               dpf|age|diabetes|
+-----------+-------+---------+-------+-------+----+------------------+---+--------+
|          6|    148|       72|     35|      0|33.6|             0.627| 50|       1|
|          1|     85|       66|     29|      0|26.6|             0.351| 31|       0|
|          8|    183|       64|      0|      0|23.3|             0.672| 32|       1|
|          1|     89|       66|     23|     94|28.1|0.1669999999999999| 21|       0|
|          0|    137|       40|     35|    168|43.1|2.2880000000000003| 33|       1|
|          5|    116|       74|      0|      0|25.6|             0.201| 30|       0|
|          3|     78|       50|     32|     88|31.0|             0.248| 26|       1|
|         10|    115|        0|      0|      0|35.3|             0.134| 29|       0|
|          2|    197|       70|     45|    543|30.5|             

**Define X & y**

In [59]:
from pyspark.ml.feature import VectorAssembler

In [60]:
diabetes.columns

['pregnancies',
 'glucose',
 'diastolic',
 'triceps',
 'insulin',
 'bmi',
 'dpf',
 'age',
 'diabetes']

In [62]:
va= VectorAssembler(inputCols=['pregnancies','glucose','diastolic','triceps','insulin','bmi','dpf','age',], outputCol='IV')

In [64]:
va.transform(diabetes).select('IV', 'diabetes').show()

+--------------------+--------+
|                  IV|diabetes|
+--------------------+--------+
|[6.0,148.0,72.0,3...|       1|
|[1.0,85.0,66.0,29...|       0|
|[8.0,183.0,64.0,0...|       1|
|[1.0,89.0,66.0,23...|       0|
|[0.0,137.0,40.0,3...|       1|
|[5.0,116.0,74.0,0...|       0|
|[3.0,78.0,50.0,32...|       1|
|[10.0,115.0,0.0,0...|       0|
|[2.0,197.0,70.0,4...|       1|
|[8.0,125.0,96.0,0...|       1|
|[4.0,110.0,92.0,0...|       0|
|[10.0,168.0,74.0,...|       1|
|[10.0,139.0,80.0,...|       0|
|[1.0,189.0,60.0,2...|       1|
|[5.0,166.0,72.0,1...|       1|
|[7.0,100.0,0.0,0....|       1|
|[0.0,118.0,84.0,4...|       1|
|[7.0,107.0,74.0,0...|       1|
|[1.0,103.0,30.0,3...|       0|
|[1.0,115.0,70.0,3...|       1|
+--------------------+--------+
only showing top 20 rows



In [65]:
modeldata= va.transform(diabetes).select('IV', 'diabetes')

In [66]:
# splitting data

train_data, test_data= modeldata.randomSplit([0.8,0.2])

In [68]:
from pyspark.ml.classification import LogisticRegression

In [69]:
lr= LogisticRegression(featuresCol='IV', labelCol='diabetes')

In [72]:
lr= lr.fit(train_data)

In [73]:
lr.coefficients

DenseVector([0.1266, 0.0339, -0.0134, -0.0051, -0.0015, 0.1074, 0.699, 0.0092])

In [74]:
y_pred= lr.transform(test_data)

In [75]:
y_pred.show()

+--------------------+--------+--------------------+--------------------+----------+
|                  IV|diabetes|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|(8,[0,1,6,7],[7.0...|       0|[3.51032626667197...|[0.97098015901758...|       0.0|
|[0.0,101.0,65.0,2...|       0|[2.97196194419159...|[0.95129126655910...|       0.0|
|[0.0,105.0,68.0,2...|       0|[3.34061146428990...|[0.96579604719897...|       0.0|
|[0.0,109.0,88.0,3...|       1|[1.59209631657375...|[0.83091083573766...|       0.0|
|[0.0,128.0,68.0,1...|       1|[0.85570358393657...|[0.70176222900737...|       0.0|
|[0.0,137.0,40.0,3...|       1|[-1.8158531397904...|[0.13993220769863...|       1.0|
|[0.0,165.0,76.0,4...|       0|[-1.1422684947688...|[0.24190410612445...|       1.0|
|[1.0,0.0,48.0,20....|       0|[6.05978561660635...|[0.99767053766689...|       0.0|
|[1.0,79.0,60.0,42...|       0|[1.32174904835313...|[0.7894725554

In [77]:
# Step 7: Evaluation

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [78]:
evaluator= MulticlassClassificationEvaluator(labelCol='diabetes', predictionCol='prediction', metricName='weightedPrecision')

In [79]:
evaluator.evaluate(y_pred)

0.8072507937562455